# Movebank API

<img align="right" src="https://anitagraser.github.io/movingpandas/pics/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=3-tech-demos/movebank.ipynb)


Source of Python wrapper with functions using Movebank's REST API: https://github.com/movebank/movebank-api-doc/blob/master/mb_Meschenmoser.py

<img align="left" src="https://www.movebank.org/cms/img/logo-movebank.png">




To use this notebook, you need a **Movebank account. Register at https://www.movebank.org**

In [ ]:
import getpass
movebank_username = getpass.getpass()

In [ ]:
movebank_password = getpass.getpass()

In [ ]:
# Author: Philipp Meschenmoser, DBVIS, Uni Konstanz
# Python wrapper with functions using Movebank's REST API to view available studies, read data and accept license terms programmatically
# Acknowledgements to Anne K. Scharf and her great moveACC package, see https://gitlab.com/anneks/moveACC

import requests
import os
import hashlib
import csv
import json
import io
from datetime import datetime, timedelta

def callMovebankAPI(params):
    # Requests Movebank API with ((param1, value1), (param2, value2),).
    # Assumes the environment variables 'mbus' (Movebank user name) and 'mbpw' (Movebank password).
    # Returns the API response as plain text.

    response = requests.get('https://www.movebank.org/movebank/service/direct-read', params=params, auth=(movebank_username, movebank_password))
    print("Request " + response.url)
    if response.status_code == 200:  # successful request
        if 'License Terms:' in str(response.content):
            # only the license terms are returned, hash and append them in a subsequent request.
            # See also
            # https://github.com/movebank/movebank-api-doc/blob/master/movebank-api.md#read-and-accept-license-terms-using-curl
            print("Has license terms")
            hash = hashlib.md5(response.content).hexdigest()
            params = params + (('license-md5', hash),)
            # also attach previous cookie:
            response = requests.get('https://www.movebank.org/movebank/service/direct-read', params=params,
                                    cookies=response.cookies, auth=(movebank_username, movebank_password))
            if response.status_code == 403:  # incorrect hash
                print("Incorrect hash")
                return ''
        return response.content.decode('utf-8')
    print(str(response.content))
    return ''


def getStudies():
    studies = callMovebankAPI((('entity_type', 'study'), ('i_can_see_data', 'true'), ('there_are_data_which_i_cannot_see', 'false')))
    if len(studies) > 0:
        # parse raw text to dicts
        studies = csv.DictReader(io.StringIO(studies), delimiter=',')
        return [s for s in studies if s['i_can_see_data'] == 'true' and s['there_are_data_which_i_cannot_see'] == 'false']
    return []


def getStudiesBySensor(studies, sensorname='GPS'):
    return [s for s in studies if sensorname in s['sensor_type_ids']]


def getIndividualsByStudy(study_id):
    individuals = callMovebankAPI((('entity_type', 'individual'), ('study_id', study_id)))
    if len(individuals) > 0:
        return list(csv.DictReader(io.StringIO(individuals), delimiter=','))
    return []


def prettyPrint(l):
    print(json.dumps(l, indent=2))


## Movebank sensor options

```
description,external_id,id,is_location_sensor,name
"","bird-ring",397,true,"Bird Ring"
"","gps",653,true,"GPS"
"","radio-transmitter",673,true,"Radio Transmitter"
"","argos-doppler-shift",82798,true,"Argos Doppler Shift"
"","natural-mark",2365682,true,"Natural Mark"
"","acceleration",2365683,false,"Acceleration"
"","solar-geolocator",3886361,true,"Solar Geolocator"
"","accessory-measurements",7842954,false,"Accessory Measurements"
"","solar-geolocator-raw",9301403,false,"Solar Geolocator Raw"
"","barometer",77740391,false,"Barometer"
"","magnetometer",77740402,false,"Magnetometer"
"","orientation",819073350,false,"Orientation"
"","solar-geolocator-twilight",914097241,false,"Solar Geolocator Twilight"

```

In [ ]:
allstudies = getStudies()

In [ ]:
gpsstudies = getStudiesBySensor(allstudies, 'GPS')
prettyPrint(gpsstudies)

In [ ]:
individuals = getIndividualsByStudy(study_id=2911040)
prettyPrint(individuals)

## Get Movebank events and create a MovingPandas trajectory

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from shapely.geometry import Point
import hvplot.pandas  # noqa

In [ ]:
STUDY_ID = 2911040
INDIVIDUAL_ID = '2911059'

In [ ]:
params = (('entity_type', 'event'), ('study_id', STUDY_ID), ('individual_id', INDIVIDUAL_ID), ('sensor_type_id', 653))
events_csv = callMovebankAPI(params)

In [ ]:
df = pd.read_csv(io.StringIO(events_csv))
df.dropna(subset=['location_long', 'location_lat'], inplace=True)

In [ ]:
gdf = gpd.GeoDataFrame(
    df.drop(['location_long', 'location_lat'], axis=1),
    crs='epsg:4326',
    geometry=[Point(xy) for xy in zip(df.location_long, df.location_lat)])

In [ ]:
gdf

In [ ]:
gdf.plot()

In [ ]:
gdf.hvplot(title=f'Movebank events of individual {INDIVIDUAL_ID}', geo=True, tiles='OSM', frame_width=700, frame_height=500)

In [ ]:
gdf['timestamp'] = pd.to_datetime(gdf['timestamp'])
gdf.set_index('timestamp', inplace=True)

In [ ]:
traj = mpd.Trajectory(gdf, 'tag_id')

In [ ]:
traj.hvplot(title=f'Trajectory of individual {INDIVIDUAL_ID}', c='speed', frame_width=700, frame_height=500, line_width=7.0, tiles='OSM', cmap='Viridis', colorbar=True)